In [1]:
from toolz import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
# %matplotlib widget


import pathlib
from lenses import lens

from collections import Counter

import re
import nltk
import pymorphy2
import fasttext.util

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

import pickle

In [2]:
nltk.download('popular')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('tagsets')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to

True

In [3]:
lmap = compose(list, map)
ltake = compose(list, take)

In [4]:
def map_df_ingredients_element(func, df):
    df = df.copy()
    df['ingredients'] = df['ingredients'].map(lambda v: lmap(func, v))
    return df

In [5]:
data_path = pathlib.Path('data')

## Загружаем датафрейм с данными

In [6]:
df = pd.read_json(data_path.joinpath('ready_dataframe.json'))
df = map_df_ingredients_element(compose(str.lower, str.strip), df)
df

,id,title,course,cuisine,ingredients
0,28195,Банановое мороженое с корицей,Выпечка и десерты,Карибская кухня,"[бананы, лимонный сок, молотая корица, ванильн..."
1,28141,Ананасово-кокосовый шербет,Выпечка и десерты,Карибская кухня,"[сахар, кокосовое молоко, корень имбиря, анана..."
2,27929,"Десерт из манго, клубники и текилы",Выпечка и десерты,Карибская кухня,"[клубника, манго, сахар, тертая цедра лайма, т..."
3,28192,Ананас с соусом из манго и рома,Выпечка и десерты,Карибская кухня,"[манго, темный ром, сок лайма, сахар, тертая ц..."
4,28463,Клубничный соус с текилой,Выпечка и десерты,Карибская кухня,"[клубника, сахарная пудра, сок лайма, текила, ..."
...,...,...,...,...,...
41433,43380,Постный борщ с фасолью,Супы,Украинская кухня,"[белая фасоль, овощной бульон, свекла, картофе..."
41434,80446,Суп энгамат,Супы,Шведская кухня,"[цветная капуста, морковь, картофель, лук-поре..."
41435,136820,Гороховый суп с блинчиками,Супы,Шведская кухня,"[горох, репчатый лук, гвоздика, свиная рулька,..."
41436,18014,Суп из брокколи и кростини с сыром бри,Супы,Шведская кухня,"[оливковое масло, репчатый лук, овощной бульон..."


## Работа с ингридиентами

Удобнее работать с листом листов:

In [7]:
ingredients = list(df['ingredients'])
ingredients[:3]

[['бананы', 'лимонный сок', 'молотая корица', 'ванильное мороженое'],
 ['сахар', 'кокосовое молоко', 'корень имбиря', 'ананас', 'сок лайма'],
 ['клубника',
  'манго',
  'сахар',
  'тертая цедра лайма',
  'текила',
  'сок лайма',
  'апельсиновый ликер']]

### Самые часто встречающиеся слова в ингридиентах

In [8]:
ingredients_counter = Counter()
for lst in ingredients:
    ingredients_counter.update(lst)
# first_ingresients, _ = zip(*ingredients_counter.most_common(10))
# first_ingresients = list(first_ingresients)
# first_ingresients
ingredients_counter.most_common(10)

[('соль', 20427),
 ('куриное яйцо', 11621),
 ('сахар', 10953),
 ('молотый черный перец', 10877),
 ('сливочное масло', 10323),
 ('пшеничная мука', 9950),
 ('чеснок', 9558),
 ('оливковое масло', 7932),
 ('репчатый лук', 7272),
 ('растительное масло', 6456)]

In [9]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(word):
    return morph.parse(word)[0].normal_form

# it = lmap(compose(tuple, nltk.word_tokenize), ingredients_counter)
# it = lens.Each().Each().modify(lemmatize)(it)
# it[:10]

### Токенизация

In [10]:
ingredients_tokenized = lens.Each().Each().modify(nltk.word_tokenize)(ingredients)
ingredients_tokenized[:4]

[[['бананы'],
  ['лимонный', 'сок'],
  ['молотая', 'корица'],
  ['ванильное', 'мороженое']],
 [['сахар'],
  ['кокосовое', 'молоко'],
  ['корень', 'имбиря'],
  ['ананас'],
  ['сок', 'лайма']],
 [['клубника'],
  ['манго'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['текила'],
  ['сок', 'лайма'],
  ['апельсиновый', 'ликер']],
 [['манго'],
  ['темный', 'ром'],
  ['сок', 'лайма'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['ананас']]]

### Стоп-слова и фильтрация

In [11]:
stopwords = nltk.corpus.stopwords.words('russian')

In [12]:
rx = re.compile(r'[a-zA-Zа-яА-Я][a-zA-Zа-яА-Я\d\-]*')

In [13]:
ingredients_filtered = (lens
    .Each().Each().modify(
        lambda tokenized: tuple(filter(
            lambda word: word not in stopwords and rx.match(word),
            tokenized))
    )(ingredients_tokenized)
)
ingredients_filtered[:4]

[[('бананы',),
  ('лимонный', 'сок'),
  ('молотая', 'корица'),
  ('ванильное', 'мороженое')],
 [('сахар',),
  ('кокосовое', 'молоко'),
  ('корень', 'имбиря'),
  ('ананас',),
  ('сок', 'лайма')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('текила',),
  ('сок', 'лайма'),
  ('апельсиновый', 'ликер')],
 [('манго',),
  ('темный', 'ром'),
  ('сок', 'лайма'),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('ананас',)]]

### Лемматизация

In [14]:
ingredients_lemmatized = lens.Each().Each().Each().modify(lemmatize)(ingredients_filtered)
ingredients_lemmatized[:4]

[[('банан',),
  ('лимонный', 'сок'),
  ('молотый', 'корица'),
  ('ванильный', 'мороженое')],
 [('сахар',),
  ('кокосовый', 'молоко'),
  ('корень', 'имбирь'),
  ('ананас',),
  ('сок', 'лайм')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('текила',),
  ('сок', 'лайм'),
  ('апельсиновый', 'ликёр')],
 [('манго',),
  ('тёмный', 'ром'),
  ('сок', 'лайм'),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('ананас',)]]

In [15]:
#ОЧЕНЬ ДОЛГО
all_words_lemmatized = Counter(lens.Each().Each().Each().collect()(ingredients_lemmatized))

In [16]:
all_words_lemmatized_list = list(all_words_lemmatized)

#Добавил UNK и PAD
all_words_lemmatized_list = ['PAD', 'UNK']+all_words_lemmatized_list
all_words_lemmatized_list.sort()
all_words_lemmatized_list[:10]
# dict(all_words.items())

['PAD',
 'UNK',
 'añejo',
 'baby',
 'baileys',
 'bambino',
 'bean',
 'biogourmet',
 'biscoff',
 'blue']

In [17]:
#ДОЛГО + много оперативки
fasttext.util.download_model('ru', if_exists='ignore')

'cc.ru.300.bin'

In [18]:
ft = fasttext.load_model('cc.ru.300.bin')
# fasttext.util.reduce_model(ft, 100)
dim = ft.get_dimension()
dim

300

### Получаем векторы для слов 

In [19]:
word_embeddings_dict = {}
for word in all_words_lemmatized_list:
    if word  == 'PAD':
        word_embeddings_dict[word] = np.concatenate((np.zeros(dim),[1, 0]))
    elif word  == 'UNK':
        word_embeddings_dict[word] = np.concatenate((np.zeros(dim),[0, 1]))
    else:
        word_embeddings_dict[word] = np.concatenate((ft.get_word_vector(word), [0, 0]))

In [20]:
first(word_embeddings_dict.items())
# np.concatenate((ft.get_word_vector(word), [0, 0])).shape

('PAD',
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

Расширяем разметрости для PAD и UNK

In [21]:
dim += 2

In [22]:
# Освободаем оперативку
del ft

### Векторы для ингредиентов

In [23]:
all_ingredients_list =  list(set(lens.Each().Each().collect()(ingredients_lemmatized)))
#Добавил UNK и PAD
all_ingredients_list = [('PAD',), ('UNK',)]+all_ingredients_list
all_ingredients_list 
ingredients_numbering_dict = {ingredient: i for i, ingredient in enumerate(all_ingredients_list)}
ltake(6, ingredients_numbering_dict.items())

[(('PAD',), 0),
 (('UNK',), 1),
 (('пряник',), 2),
 (('засахарить', 'имбирь'), 3),
 (('филе', 'толстолобик'), 4),
 (('сладкий', 'минь', 'перец'), 5)]

Для получения вектора для ингридиента просто суммируем векторы для каждого слова в этом ингридиенте:

In [24]:
ingredients_embeddings_dict = {}
for ingr in all_ingredients_list:
    ingredients_embeddings_dict[ingr] = np.zeros(dim)
    for word in ingr:
#         ingredients_embeddings_dict[ingr] += ft.get_word_vector(word) 
        ingredients_embeddings_dict[ingr] += word_embeddings_dict[word] # Должно заработать, когда 
                                                                         # ингредиенты будут лемматизированны
ltake(10, ingredients_embeddings_dict.items())[3]


(('засахарить', 'имбирь'),
 array([ 2.02463828e-02,  6.49796426e-03,  4.99049388e-03,  9.85940685e-02,
         1.16182785e-01, -1.06950356e-01,  1.09567110e-01,  1.94525160e-01,
        -1.00431688e-01, -1.12146374e-01, -2.45975098e-02, -2.01643603e-02,
        -1.99245382e-03,  2.52078529e-02,  6.38375496e-02,  4.35856008e-02,
         2.80356780e-02, -9.33141336e-02, -1.05585545e-01, -7.25313704e-02,
         5.10598943e-02,  1.64475208e-02, -2.46114535e-02, -2.08923612e-02,
        -9.87240486e-02, -4.03482569e-02, -2.18254719e-02, -2.25545093e-03,
         1.13083106e-02,  9.47969705e-02, -8.26240890e-02,  5.54792032e-02,
        -1.30902492e-02,  2.65804306e-03,  1.30749475e-02, -1.41853206e-01,
         4.92830155e-03, -1.09910091e-01, -1.36481021e-02, -4.06353753e-02,
         1.53309301e-01, -5.05606355e-02,  9.02719945e-02,  2.27633966e-02,
         9.52837504e-02,  4.21115328e-02,  4.19562442e-02, -3.32674626e-02,
         1.61473773e-01,  8.56094398e-02,  7.30349105e-02,  9

Наконец, для каждого рецепта возьмем его векторизацию как сумму векторов его ингридиентов

In [34]:
recipe_embedding_sum = [
    sum(ingredients_embeddings_dict[ingredient]
        for ingredient in ingredient_list) 
    for ingredient_list in ingredients_lemmatized
]
len(recipe_embedding_sum)

41438

In [35]:
recipe_embedding_avg = [
    sum(ingredients_embeddings_dict[ingredient]
        for ingredient in ingredient_list)/len(ingredient_list) 
    for ingredient_list in ingredients_lemmatized
]
len(recipe_embedding_avg)

41438

## Данные для нейросети

Всевозможные метки (course):

In [36]:
all_courses = {name: num for num, name in enumerate(df['course'][~df['course'].duplicated()])}
all_courses

{'Выпечка и десерты': 0,
 'Завтраки': 1,
 'Закуски': 2,
 'Напитки': 3,
 'Основные блюда': 4,
 'Паста и пицца': 5,
 'Салаты': 6,
 'Соусы и маринады': 7,
 'Супы': 8}

Группируем кухни:

In [37]:
df_cuisines = pd.read_excel(data_path.joinpath('Сортировка_кухонь.xlsx'))
cuisines_groups = {}
for column in df_cuisines.columns:
    series = df_cuisines[column]
    cuisines_groups[str(column).strip()] = set(map(str.strip, list(series[series.notnull()])))
del cuisines_groups['Unnamed: 1']
inverse_cuisines_groups = {cuisine: cuisine_group
                           for (cuisine_group, cuisines) in cuisines_groups.items()
                           for cuisine in cuisines}
inverse_cuisines_groups

{'Швейцарская кухня': 'Европейская',
 'Башкирская кухня': 'Центральноазиатская',
 'Хорватская кухня': 'Балканская',
 'Авторская кухня': 'Неклассифицируемое',
 'Арабская кухня': 'Ближний Восток',
 'Португальская кухня': 'Европейская',
 'Датская кухня': 'Скандинавская',
 'Скандинавская кухня': 'Скандинавская',
 'Тайская кухня': 'Юго-Восточная Азия',
 'Румынская кухня': 'Балканская',
 'Турецкая кухня': 'Балканская',
 'Таджикская кухня': 'Центральноазиатская',
 'Мировая кухня': 'Неклассифицируемое',
 'Афганская кухня': 'Центральноазиатская',
 'Бельгийская кухня': 'Европейская',
 'Исландская кухня': 'Скандинавская',
 'Югославская кухня': 'Балканская',
 'Татарская кухня': 'Центральноазиатская',
 'Камбоджийская кухня': 'Юго-Восточная Азия',
 'Болгарская кухня': 'Балканская',
 'Итальянская кухня': 'Европейская',
 'Кубинская кухня': 'Североамериканская',
 'Марокканская кухня': 'Африканская',
 'Австрийская кухня': 'Европейская',
 'Китайская кухня': 'Восточноазиатская',
 'Чешская кухня': 'Европей

In [38]:
cuisines_set = set(inverse_cuisines_groups.values())
# print(cuisines_set)
all_cuisines = {name: num for num, name in enumerate(cuisines_set)}
all_cuisines

{'Юго-Восточная Азия': 0,
 'Кавказская': 1,
 'Славянская': 2,
 'Европейская': 3,
 'Ближний Восток': 4,
 'Североамериканская': 5,
 'Африканская': 6,
 'Центральноазиатская': 7,
 'Восточноазиатская': 8,
 'Скандинавская': 9,
 'Южноамериканская': 10,
 'Балканская': 11,
 'Неклассифицируемое': 12}

### Делаем tf-idf

In [39]:
recipe_series = pd.Series(index=df['id'], data=ingredients_lemmatized)
recipe_series

id
28195     [(банан,), (лимонный, сок), (молотый, корица),...
28141     [(сахар,), (кокосовый, молоко), (корень, имбир...
27929     [(клубника,), (манго,), (сахар,), (тёртый, цед...
28192     [(манго,), (тёмный, ром), (сок, лайм), (сахар,...
28463     [(клубника,), (сахарный, пудра), (сок, лайм), ...
                                ...                        
43380     [(белый, фасоль), (овощной, бульон), (свёкла,)...
80446     [(цветной, капуста), (морковь,), (картофель,),...
136820    [(горох,), (репчатый, лук), (гвоздик,), (свино...
18014     [(оливковый, масло), (репчатый, лук), (овощной...
49530     [(картофель,), (репчатый, лук), (мясной, фарш)...
Length: 41438, dtype: object

In [40]:
try:
    del tf
finally:
    tf = pd.DataFrame(index=df['id'].rename('recipe_id'), 
                      columns=all_ingredients_list,
                      data=0,
                      dtype=np.float64)
for index, count in tf.iterrows():
    for ingredient in recipe_series.loc[index]:
        count[ingredient] += 1
    count /= len(recipe_series.loc[index])
tf

,"(PAD,)","(UNK,)","(пряник,)","(засахарить, имбирь)","(филе, толстолобик)","(сладкий, минь, перец)","(конфета, m, m, s)","(мясо, кость)","(тёмный, ром)","(молотый, сушёный, розмарин)",...,"(рубленый, шнитта-лука)","(груша, конференц)","(сыр, хаварти)","(утиный, бульон)","(севильский, апельсин)","(жасминовый, тайский, рис)","(мариновать, красный, тоф)","(клюква,)","(специя, птица)","(пальмовый, масло)"
recipe_id,,,,,,,,,,,,,,,,,,,,,
28195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
idf = dict(zip(tf.columns, iterate(lambda _: 1, 1)))
for recipe in recipe_series.values:
    for ingredient in recipe:
        idf[ingredient] += 1
idf = pd.Series(index=pd.Index(idf.keys(), tupleize_cols=False), data=idf.values(), dtype=np.float64)
idf /= len(tf.index)
idf = -idf.apply(np.log)
idf

(PAD,)                        10.631954
(UNK,)                        10.631954
(пряник,)                      8.840194
(засахарить, имбирь)           6.943074
(филе, толстолобик)            9.938806
                                ...    
(жасминовый, тайский, рис)     9.938806
(мариновать, красный, тоф)     9.533341
(клюква,)                      5.959125
(специя, птица)                7.687515
(пальмовый, масло)             8.840194
Length: 4406, dtype: float64

In [42]:
try:
    del tfidf
finally:
    tfidf = tf*idf
tfidf

,"(PAD,)","(UNK,)","(пряник,)","(засахарить, имбирь)","(филе, толстолобик)","(сладкий, минь, перец)","(конфета, m, m, s)","(мясо, кость)","(тёмный, ром)","(молотый, сушёный, розмарин)",...,"(рубленый, шнитта-лука)","(груша, конференц)","(сыр, хаварти)","(утиный, бульон)","(севильский, апельсин)","(жасминовый, тайский, рис)","(мариновать, красный, тоф)","(клюква,)","(специя, птица)","(пальмовый, масло)"
recipe_id,,,,,,,,,,,,,,,,,,,,,
28195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.020182,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Все нужные данные переводим в pickle

In [44]:
df_for_learning = pd.DataFrame(data={
    'id': df['id'],
    'recipe_embedding_sum': recipe_embedding_sum,
    'recipe_embedding_avg': recipe_embedding_avg,
    'course_mark': [all_courses[course] for course in df['course']],
    'cuisine_group_mark': [all_cuisines[inverse_cuisines_groups[cuisine]] for cuisine in df['cuisine']]
})
df_for_learning

,id,recipe_embedding_sum,recipe_embedding_avg,course_mark,cuisine_group_mark
0,28195,"[0.611611146479845, -0.0743011748418212, -0.09...","[0.15290278661996126, -0.0185752937104553, -0....",0,10
1,28141,"[0.8909059994039126, -0.11706455610692501, -0....","[0.17818119988078251, -0.023412911221385002, -...",0,10
2,27929,"[1.08976399153471, -0.4261779775843024, -0.389...","[0.15568057021924428, -0.06088256822632892, -0...",0,10
3,28192,"[1.1749160811305046, -0.5175759345293045, -0.3...","[0.1958193468550841, -0.08626265575488408, -0....",0,10
4,28463,"[0.6190221486613154, -0.14342674519866705, -0....","[0.12380442973226309, -0.02868534903973341, -0...",0,10
...,...,...,...,...,...
41433,43380,"[1.3148583127185702, -0.05140630202367902, -0....","[0.08765722084790468, -0.003427086801578601, -...",8,2
41434,80446,"[1.0693077351897955, 0.24955729965586215, -0.1...","[0.07128718234598637, 0.01663715331039081, -0....",8,9
41435,136820,"[0.7934096870012581, 0.5892392117530107, -0.44...","[0.056672120500089865, 0.042088515125215054, -...",8,9
41436,18014,"[0.834478304721415, 0.4054728071205318, 0.2355...","[0.07586166406558319, 0.03686116428368471, 0.0...",8,9


In [45]:
dict_to_pickle = {
    'df_for_learning': df_for_learning,
    'all_courses': all_courses,
    'all_cuisines': all_cuisines,
    'word_embeddings_dict': word_embeddings_dict,
    'ingredients_numbering_dict': ingredients_numbering_dict,
    'dim': dim,
    'ingredients_lemmatized': ingredients_lemmatized,
    'all_ingredients_list': all_ingredients_list,
    'ingredients_embeddings_dict': ingredients_embeddings_dict,
    # 'tfidf': tfidf
}
with data_path.joinpath('processed_data_dict.pkl').open(mode='wb') as f:
    pickle.dump(dict_to_pickle, f)